In [374]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

os.chdir(r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\Data Cleaning\22 July 2025\2019")
os.getcwd()

# Filepaths for the datasets
filepaths = {
    'df1': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2019 nss 77th round\AIDIS 2019 All datasets\Visit1  Level - 04 (Block 4) - Household characteristics.dta",
    
    'df2': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2019 nss 77th round\AIDIS 2019 All datasets\Visit1  Level - 12 (Block 11a) - Financial assets including receivables (excluding shares) owned.dta",

    'df3': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2019 nss 77th round\AIDIS 2019 All datasets\Visit1  Level - 13 (Block 11b) - Investments in share and related instruments owned in co-operative societies & companies..dta",

    'df4': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2019 nss 77th round\AIDIS 2019 All datasets\Visit1  Level - 14 (Block 12) - particulars of cash loans payable to institutional, non-institutional agencies.dta",

    'df5': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2019 nss 77th round\AIDIS 2019 All datasets\Visit1  Level - 07 (Block 6) - Buildings and other constructions owned by the household.dta",

    'df6': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2019 nss 77th round\AIDIS 2019 All datasets\Visit1  Level - 06 (Block 5.2) -URBAN LAND owned by the household.dta",

    'df7': r"C:\Users\siddu\Downloads\US Replication Folder\Trying for India\AIDIS Data\AIDIS 2019 nss 77th round\AIDIS 2019 All datasets\Visit1  Level - 05 (Block 5pt1) - RURAL LAND owned by the household .dta"
}



In [375]:
# Load dataframes
dataframes = {name: pd.read_stata(path) for name, path in filepaths.items()}

# Print unique HHIDs in first seven datasets
for i in range(1, 8):
    df = dataframes[f'df{i}']
    unique_hhids = set(df['HHID'].dropna())
    print(f"Unique HHIDs in df{i}: {len(unique_hhids)}")

# Compute common HHIDs: intersection of first five & union of df6, df7
common_hhids = set.intersection(*[
    set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(1, 6)
]) & set.union(*[
    set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(6, 8)
])

print(f"Common HHIDs across: {len(common_hhids)}")

# Filter each dataframe to keep only rows with HHID in common_hhids
for name in dataframes:
    df = dataframes[name]
    df_filtered = df[df['HHID'].isin(common_hhids)].copy()
    dataframes[name] = df_filtered  # Replace with filtered version

# Optional: Print how many rows remain in each
for name, df in dataframes.items():
    print(f"{name}: {len(df)} rows after filtering")

# Reset index
for key in dataframes:
    dataframes[key] = dataframes[key].reset_index(drop=True)

# Convert all columns to numeric except 'HHID' for each dataframe
for i in range(1, 8):
    df = dataframes[f'df{i}']
    cols_to_convert = df.columns.difference(['HHID'])
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
    dataframes[f'df{i}'] = df  # Update dictionary

# Unpack if needed
df1, df2, df3, df4, df5, df6, df7 = [dataframes[f'df{i}'] for i in range(1, 8)]


Unique HHIDs in df1: 116461
Unique HHIDs in df2: 115943
Unique HHIDs in df3: 1615
Unique HHIDs in df4: 72938
Unique HHIDs in df5: 97572
Unique HHIDs in df6: 31206
Unique HHIDs in df7: 71400
Common HHIDs across: 1123
df1: 1123 rows after filtering
df2: 6434 rows after filtering
df3: 2297 rows after filtering
df4: 3571 rows after filtering
df5: 3048 rows after filtering
df6: 1111 rows after filtering
df7: 2559 rows after filtering


In [376]:
# df1: Keep relevant columns
df1 = df1[['HHID', 'State', 'b4q10dot5', 'MLT']].copy()

# Checking for missing values in MCE
df1['b4q10dot5'].isna().sum()


# df2 is Fin Asset- 2019
df2 = df2[['HHID', 'b11aq1', 'b11aq3', 'b11aq6']].copy()
df2 = df2[df2['b11aq1'].isin([1,2,3,4,5,6,7,8,9,10,11,13])] 

# df3: Keep relevant columns
df3 = df3[['HHID', 'b11bq1', 'b11bq3', 'b11bq6']].copy()
df3 = df3[df3['b11bq1'] == 5] 

# df4: Keep relevant columns
df4 = df4[['HHID', 'b12q1', 'b12q5', 'b12q14', 'b12q15']].copy()
df4 = df4[df4['b12q5'].isin([1,2,3,4,5,6,8,10,13])] 

# df5: Keep only rows where b6q3 == 10
df5 = df5[['HHID', 'b6q3', 'b6q5']].copy()
df5 = df5[df5['b6q3'] == 10].copy()

# df6: Keep only rows where b5pt2q1 == 99
df6 = df6[['HHID', 'b5pt2q1', 'b5pt2q5']].copy()
df6 = df6[df6['b5pt2q1'] == 99].copy()

# df7: Keep only rows where b5pt1q1 == 99
df7 = df7[['HHID', 'b5pt1q1', 'b5pt1q5']].copy()
df7 = df7[df7['b5pt1q1'] == 99].copy()

In [377]:
# Renaming, creating, dropping columns 

df1  = df1.rename(columns={
    'b4q10dot5': 'MCE'
})

df2  = df2.rename(columns={
    'b11aq1': 'Fin_Asset_Serial',
    'b11aq3': 'Fin_Asset_val1',
    'b11aq6': 'Fin_Asset_val2'
})

df3  = df3.rename(columns={
    'b11bq1': 'Share_Asset_serial',
    'b11bq3': 'Share_Asset_val1',
    'b11bq6': 'Share_Asset_val2'
})

df4  = df4.rename(columns={
    'b12q1': 'Liability_serial',
    'b12q5': 'Credit_Agency',
    'b12q14': 'Liability_val1',
    'b12q15': 'Liability_val2'
})

df5  = df5.rename(columns={
    'b6q3': 'Building_serial',
    'b6q5': 'Building_value'
})

df6  = df6.rename(columns={
    'b5pt2q1': 'Urban_land_serial',
    'b5pt2q5': 'Urban_land_value'
})

df7  = df7.rename(columns={
    'b5pt1q1': 'Rural_land_serial',
    'b5pt1q5': 'Rural_land_value',
})

#df1['MCE'] = df1['MCE']/1000
df1['Weight'] = df1['MLT']/200

# weight

In [378]:
df1.columns

Index(['HHID', 'State', 'MCE', 'MLT', 'Weight'], dtype='object')

In [379]:
df2['Fin_Asset_Value'] = np.where(
    df2['Fin_Asset_Serial'] == 1,
    df2['Fin_Asset_val1'],
    df2['Fin_Asset_val1'].combine_first(df2['Fin_Asset_val2'])
)
df2['Fin_Asset_Value'] = pd.Series(df2['Fin_Asset_Value']).fillna(0)

# df3 (Share Asset value)

df3['Share_Asset_Value'] = df3['Share_Asset_val1'].combine_first(df3['Share_Asset_val2'])
df3['Share_Asset_Value'] = df3['Share_Asset_Value'].fillna(0)

# df4 (Loan value)
df4['Liability_value'] = df4['Liability_val1'].combine_first(df4['Liability_val2'])
df4['Liability_value'] = df4['Liability_value'].fillna(0)

# df5 (Buildings value)
df5['Building_value'] = df5['Building_value'].fillna(0)

# df6 (Urban land value)
df6['Urban_land_value'] = df6['Urban_land_value'].fillna(0)

# df7 (Rural land value)
df7['Rural_land_value'] = df7['Rural_land_value'].fillna(0)


# Drop unnecessary variables
df1.drop(['MLT'], axis=1, inplace=True)
df2.drop(['Fin_Asset_val1', 'Fin_Asset_val2'], axis=1, inplace=True)
df3.drop(['Share_Asset_val1', 'Share_Asset_val2'], axis=1, inplace=True)
df4.drop(['Liability_serial', 'Liability_val1', 'Liability_val2'], axis=1, inplace=True)
df5.drop(['Building_serial'], axis=1, inplace=True)
df6.drop(['Urban_land_serial'], axis=1, inplace=True)
df7.drop(['Rural_land_serial'], axis=1, inplace=True)


# Save to Stata
df1.to_stata("df1_2019.dta", write_index=False)
df2.to_stata("df2_2019.dta", write_index=False)
df3.to_stata("df3_2019.dta", write_index=False)
df4.to_stata("df4_2019.dta", write_index=False)
df5.to_stata("df5_2019.dta", write_index=False)
df6.to_stata("df6_2019.dta", write_index=False)
df7.to_stata("df7_2019.dta", write_index=False)      

In [380]:
for i in range(1, 8):
    globals()[f"df{i}"] = globals()[f"df{i}"].reset_index(drop=True)


In [381]:
print(df2['Fin_Asset_Serial'].value_counts(dropna=False))

# Step 2: Group by HHID and sum Fin_Asset_Value
summary_rows = df2.groupby('HHID', as_index=False)['Fin_Asset_Value'].sum()
summary_rows['Fin_Asset_Serial'] = 999

# Step 3: Add other columns with NaN or defaults
for col in df2.columns:
    if col not in ['HHID', 'Fin_Asset_Serial', 'Fin_Asset_Value']:
        summary_rows[col] = np.nan

# Step 4: Concatenate the new rows
df2_mod = pd.concat([df2, summary_rows], ignore_index=True)

# Step 5: Sort
df2_mod = df2_mod.sort_values(by=['HHID', 'Fin_Asset_Serial']).reset_index(drop=True)

# Step 6: Save
df2_mod.to_stata("mergedforu.dta", write_index=False)

print(df2_mod.head())
print(df2_mod['HHID'].nunique())

df2_mod['Fin_Asset_Serial'].value_counts(dropna=False)

# Modify serials and labels, if any

# df2_mod
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([1]), 'Fin_Asset_Serial'] = 31
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([2,3,4,7,8,9]), 'Fin_Asset_Serial'] = 32
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([13]), 'Fin_Asset_Serial'] = 34
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([10,11]), 'Fin_Asset_Serial'] = 35
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([5,6]), 'Fin_Asset_Serial'] = 36
df2_mod.loc[df2_mod['Fin_Asset_Serial'].isin([999]), 'Fin_Asset_Serial'] = 37

df2_mod = df2_mod.reset_index(drop=True)

# Fin Asset Mapping

Fin_Asset_map = {

    31.0: 'Currency',
    32.0: 'Deposits',
    34.0: 'Life Insurance fund',
    35.0: 'Provident and Pension fund',
    36.0: 'Claims on Government',
    37.0: 'Total financial assets (other thanshares and related instruments)'
}

df2_mod = df2_mod.groupby(['HHID', 'Fin_Asset_Serial'], as_index=False).agg({
    'Fin_Asset_Value': 'sum'  
})

df2_mod = df2_mod.reset_index(drop=True)

df2_mod.to_stata("mergedforu.dta", write_index=False)

df2_mod['Fin_Asset_Serial'].value_counts()

df2 = df2_mod.copy()

Fin_Asset_Serial
1     1116
3     1086
13     505
9      498
5      234
4      164
10     148
7       85
2       80
11      73
8       32
6       16
Name: count, dtype: int64
        HHID  Fin_Asset_Serial  Fin_Asset_Value
0  501171011                 1           3000.0
1  501171011                 3          20000.0
2  501171011               999          23000.0
3  501171031                 1           4000.0
4  501171031                 3          60000.0
1122


In [382]:
df3['Share_Asset_serial'].value_counts()

df3 = df3.reset_index(drop=True)

# Share_Asset_Map

# Mapping dictionary using float keys (since your column is float64)
Share_Asset_Map = {
    5.0: 'Shares and Debentures'
}


In [383]:
print(df4['Credit_Agency'].value_counts(dropna=False))

# Step 2: Group by HHID and sum Liability_value
summary_rows_liab = df4.groupby('HHID', as_index=False)['Liability_value'].sum()
summary_rows_liab['Credit_Agency'] = 999

# Step 3: Add other columns with NaN or defaults
for col in df4.columns:
    if col not in ['HHID', 'Credit_Agency', 'Liability_value']:
        summary_rows_liab[col] = np.nan

# Step 4: Concatenate the new rows
df4_mod = pd.concat([df4, summary_rows_liab], ignore_index=True)

# Step 5: Sort
df4_mod = df4_mod.sort_values(by=['HHID', 'Credit_Agency']).reset_index(drop=True)

# Step 6: Save
df4_mod.to_stata("mergedforu_liab.dta", write_index=False)

print(df4_mod.head())
print(df4_mod['HHID'].nunique())

# df4_mod

df4_mod.loc[df4_mod['Credit_Agency'].between(1, 4), 'Credit_Agency'] = 91
df4_mod.loc[df4_mod['Credit_Agency'].isin([5, 6, 8, 10, 13]), 'Credit_Agency'] = 92

df4_mod = df4_mod.reset_index(drop=True)

# Liability Map

Credit_Agency_Map = {

    91.0: 'Bank advances',
    92.0: 'Non-banking loans and advances',
    999.0: 'Total Institutional Liabilities'
}

df4_mod = df4_mod.groupby(['HHID', 'Credit_Agency'], as_index=False).agg({
    'Liability_value': 'sum'  
})

df4_mod = df4_mod.reset_index(drop=True)

df4_mod.to_stata("mergedforu_liab.dta", write_index=False)

df4_mod['Credit_Agency'].value_counts()

df4 = df4_mod.copy()

Credit_Agency
1.0     750
3.0     501
4.0     221
10.0     79
8.0      53
2.0      51
5.0      18
13.0     12
6.0       5
Name: count, dtype: int64
        HHID  Credit_Agency  Liability_value
0  501171011            3.0          10000.0
1  501171011          999.0          10000.0
2  501171031            3.0         100000.0
3  501171031          999.0         100000.0
4  501171041            3.0         500000.0
1004


In [384]:
df6.head()

,HHID,Urban_land_value
0,532614011,500000.0
1,532601011,900000.0
2,536206031,700000.0
3,536101021,780000.0
4,554221021,1000000.0


In [385]:
dataframes['df1'] = df1
dataframes['df2'] = df2
dataframes['df3'] = df3
dataframes['df4'] = df4
dataframes['df5'] = df5
dataframes['df6'] = df6
dataframes['df7'] = df7

In [386]:
# Print unique HHIDs in first seven datasets
for i in range(1, 8):
    df = dataframes[f'df{i}']
    unique_hhids = set(df['HHID'].dropna())
    print(f"Unique HHIDs in df{i}: {len(unique_hhids)}")

# Compute common HHIDs: intersection of first five & union of df6, df7
common_hhids = set.intersection(*[
    set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(1, 6)
]) & set.union(*[
    set(dataframes[f'df{i}']['HHID'].dropna()) for i in range(6, 8)
])

print(f"Common HHIDs across: {len(common_hhids)}")

# Filter each dataframe to keep only rows with HHID in common_hhids
for name in dataframes:
    df = dataframes[name]
    df_filtered = df[df['HHID'].isin(common_hhids)].copy()
    dataframes[name] = df_filtered  # Replace with filtered version

# Optional: Print how many rows remain in each
for name, df in dataframes.items():
    print(f"{name}: {len(df)} rows after filtering")

# Reset index
for key in dataframes:
    dataframes[key] = dataframes[key].reset_index(drop=True)

# Convert all columns to numeric except 'HHID' for each dataframe
for i in range(1, 8):
    df = dataframes[f'df{i}']
    cols_to_convert = df.columns.difference(['HHID'])
    df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')
    dataframes[f'df{i}'] = df  # Update dictionary

# Unpack if needed
df1, df2, df3, df4, df5, df6, df7 = [dataframes[f'df{i}'] for i in range(1, 8)]

Unique HHIDs in df1: 1123
Unique HHIDs in df2: 1122
Unique HHIDs in df3: 1123
Unique HHIDs in df4: 1004
Unique HHIDs in df5: 1123
Unique HHIDs in df6: 467
Unique HHIDs in df7: 793
Common HHIDs across: 1004
df1: 1004 rows after filtering
df2: 3873 rows after filtering
df3: 1004 rows after filtering
df4: 2084 rows after filtering
df5: 1004 rows after filtering
df6: 430 rows after filtering
df7: 703 rows after filtering


In [387]:
# Merging all datasets
from functools import reduce

dfs = [df1, df2, df3, df4, df5, df6, df7]

merged_df = reduce(lambda left, right: pd.merge(left, right, on='HHID', how='outer'), dfs)

merged_df.to_stata("merged.dta", write_index=False)

unique_hhids = set(merged_df['HHID'].dropna())
print(f"Unique HHIDs: {len(unique_hhids)}")


# Land value
merged_df['Land_value'] = merged_df[['Urban_land_value', 'Rural_land_value']].sum(axis=1, min_count=1).fillna(0)

# Real Estate Value

merged_df['Real Estate'] = merged_df['Building_value'] + merged_df['Land_value']

merged_df.to_stata("merged1.dta", write_index=False)

merged_df1 = merged_df.copy()

merged_df1.columns

merged_df1.drop(['Building_value', 'Urban_land_value',
                'Rural_land_value', 'Land_value'
                ], axis=1, inplace=True)

merged_df1.to_stata("merged1.dta", write_index=False)

merged_df1.columns

# Check for duplicates
group_cols = ['HHID', 'Credit_Agency', 'Fin_Asset_Serial', 'Share_Asset_serial']
dup_check = merged_df1.duplicated(subset=group_cols)
print(f"Number of duplicates: {dup_check.sum()}")

unique_count = merged_df1.loc[merged_df1['Fin_Asset_Serial'] == 37, 'HHID'].nunique()
print(f"Unique HHIDs with Fin_Asset_Serial = 37: {unique_count}")

unique_count = merged_df1.loc[merged_df1['Share_Asset_serial'] == 5, 'HHID'].nunique()
print(f"Unique HHIDs with Share_Asset_serial = 5: {unique_count}")

unique_count = merged_df1.loc[merged_df1['Credit_Agency'] == 999, 'HHID'].nunique()
print(f"Unique HHIDs with Credit_Agency = 999: {unique_count}")


Unique HHIDs: 1004
Number of duplicates: 0
Unique HHIDs with Fin_Asset_Serial = 37: 1004
Unique HHIDs with Share_Asset_serial = 5: 1004
Unique HHIDs with Credit_Agency = 999: 1004


C:\Users\siddu\AppData\Local\Temp\ipykernel_46848\1925433272.py:21: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  merged_df.to_stata("merged1.dta", write_index=False)
C:\Users\siddu\AppData\Local\Temp\ipykernel_46848\1925433272.py:31: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  merged_df1.to_stata("merged1.dta", write_index=False)


In [388]:
cols = ['HHID', 'State', 'MCE', 'Fin_Asset_Serial', 'Share_Asset_serial',
        'Credit_Agency', 'Share_Asset_Value', 'Fin_Asset_Value',
        'Liability_value', 'Real Estate', 'Weight']

# Create summary dataframe
summary = pd.DataFrame({
    'Column': cols,
    'Missing_Values': [merged_df1[col].isna().sum() for col in cols],
    'Zero_Values': [(merged_df1[col] == 0).sum() for col in cols]
})

print(summary)


# Unique HHIDs

unique_hhids = set(merged_df1['HHID'].dropna())
print(f"Unique HHIDs: {len(unique_hhids)}")

merged_df1.to_stata("merged22.dta", write_index=False)


                Column  Missing_Values  Zero_Values
0                 HHID               0            0
1                State               0            0
2                  MCE               0            0
3     Fin_Asset_Serial               0            0
4   Share_Asset_serial               0            0
5        Credit_Agency               0            0
6    Share_Asset_Value               0            0
7      Fin_Asset_Value               0            4
8      Liability_value               0          112
9          Real Estate               0            0
10              Weight               0            0
Unique HHIDs: 1004


C:\Users\siddu\AppData\Local\Temp\ipykernel_46848\724980383.py:20: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  merged_df1.to_stata("merged22.dta", write_index=False)


In [389]:
merged_df3 = merged_df1.copy()

# Mapping

In [390]:
# Create the new column
merged_df3['Fin_Asset'] = merged_df3['Fin_Asset_Serial'].map(Fin_Asset_map)
merged_df3['Share_Asset'] = merged_df3['Share_Asset_serial'].map(Share_Asset_Map)
merged_df3['Liability'] = merged_df3['Credit_Agency'].map(Credit_Agency_Map)

In [391]:
# Unique HHIDs

unique_hhids = set(merged_df3['HHID'].dropna())
print(f"Unique HHIDs: {len(unique_hhids)}")

Unique HHIDs: 1004


In [392]:
# HHIDs satisfying each condition
hhid_fin = set(merged_df3.loc[merged_df3['Fin_Asset_Serial'] == 37, 'HHID'])
hhid_share = set(merged_df3.loc[merged_df3['Share_Asset_serial'] == 5, 'HHID'])
hhid_credit = set(merged_df3.loc[merged_df3['Credit_Agency'] == 999, 'HHID'])

# Intersection: HHIDs present in all three
common_hhids = hhid_fin & hhid_share & hhid_credit

# Keep only those rows
filtered_df = merged_df3[merged_df3['HHID'].isin(common_hhids)].copy()

# Print how many unique HHIDs remain
print(f"Unique HHIDs present in all three groups: {len(common_hhids)}")

#filtered_df.drop(['Fin_Asset', 'Share_Asset', 'Liability'], axis=1, inplace=True)

#filtered_df.to_stata("final(1).dta", write_index=False)

Unique HHIDs present in all three groups: 1004


In [393]:
filtered_df.columns

Index(['HHID', 'State', 'MCE', 'Weight', 'Fin_Asset_Serial', 'Fin_Asset_Value',
       'Share_Asset_serial', 'Share_Asset_Value', 'Credit_Agency',
       'Liability_value', 'Real Estate', 'Fin_Asset', 'Share_Asset',
       'Liability'],
      dtype='object')

In [394]:
# Sanity checks if grouping worked correctly

print(sorted(filtered_df['Fin_Asset_Serial'].unique()))
#print(filtered_df['Fin_Asset_Serial'].value_counts())

print(sorted(filtered_df['Share_Asset_serial'].unique()))
#print(filtered_df['Share_Asset_serial'].value_counts())

print(sorted(filtered_df['Credit_Agency'].unique()))
#print(filtered_df['Credit_Agency'].value_counts())

[31, 32, 34, 35, 36, 37]
[5]
[91.0, 92.0, 999.0]


In [395]:
df_grouped1 = filtered_df.copy()

In [396]:
# Step 1: Filter the relevant rows satisfying your condition
wealth_rows = df_grouped1[
    (df_grouped1['Credit_Agency'] == 999) &
    (df_grouped1['Fin_Asset_Serial'] == 37) &
    (df_grouped1['Share_Asset_serial'] == 5)
].copy()

# Step 2: Compute wealth for these rows
wealth_rows['Gross wealth'] = (
    wealth_rows['Fin_Asset_Value'] +
    wealth_rows['Share_Asset_Value'] +
    wealth_rows['Real Estate']
)

wealth_rows['Total Financial Assets'] = (
    wealth_rows['Fin_Asset_Value']+
    wealth_rows['Share_Asset_Value']
)

wealth_rows['Total Institutional liabilities'] = wealth_rows['Liability_value']

# Step 3: Keep only HHID and wealth columns
wealth_per_hhid = wealth_rows[['HHID', 'Gross wealth', 'Total Financial Assets', 'Total Institutional liabilities']]

# Step 4: Merge wealth back to the full dataset based on HHID
df_grouped1 = df_grouped1.merge(wealth_per_hhid, on='HHID', how='left')

# Step 5: View result
print(df_grouped1[['HHID', 'Gross wealth']].drop_duplicates().head())

df_grouped1.to_stata("final(25).dta", write_index=False)

         HHID  Gross wealth
0   501171011     1523500.0
6   501171031     5329000.0
14  501171041     2693000.0
26  501174041     3388000.0
32  501271011     2013000.0


C:\Users\siddu\AppData\Local\Temp\ipykernel_46848\3101814955.py:31: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  df_grouped1.to_stata("final(25).dta", write_index=False)


In [397]:
df_grouped2 =  df_grouped1.copy()

In [398]:
import numpy as np

# Initialize columns with NaN
df_grouped2['Asset_serial'] = np.nan
df_grouped2['Asset_Value'] = np.nan
df_grouped2['Asset_Name'] = np.nan

group_cols = ['HHID', 'Credit_Agency', 'Fin_Asset_Serial', 'Share_Asset_serial']

df_grouped2['HHID'] = df_grouped2['HHID'].astype(str)

#df_grouped2.to_stata("final(26).dta", write_index=False)


In [399]:
df_grouped2.columns

Index(['HHID', 'State', 'MCE', 'Weight', 'Fin_Asset_Serial', 'Fin_Asset_Value',
       'Share_Asset_serial', 'Share_Asset_Value', 'Credit_Agency',
       'Liability_value', 'Real Estate', 'Fin_Asset', 'Share_Asset',
       'Liability', 'Gross wealth', 'Total Financial Assets',
       'Total Institutional liabilities', 'Asset_serial', 'Asset_Value',
       'Asset_Name'],
      dtype='object')

In [400]:
# Empty list to store new rows
rows = []

# Loop over each unique group
for _, group in df_grouped2.groupby(group_cols):

    # Get first row for copying other columns
    first_row = group.iloc[0].to_dict()

    # 1. Financial Asset Row
    if not np.isnan(first_row['Fin_Asset_Value']):
        fa_row = first_row.copy()
        fa_row['Asset_serial'] = first_row['Fin_Asset_Serial']
        fa_row['Asset_Value'] = first_row['Fin_Asset_Value']
        fa_row['Asset_Name'] = first_row['Fin_Asset']
        rows.append(fa_row)

    # 2. Share Asset Row
    if not np.isnan(first_row['Share_Asset_Value']):
        sa_row = first_row.copy()
        sa_row['Asset_serial'] = 200 + first_row['Share_Asset_serial']  # As you specified
        sa_row['Asset_Value'] = first_row['Share_Asset_Value']
        sa_row['Asset_Name'] = first_row['Share_Asset']
        rows.append(sa_row)

    # 3. Real Estate Row
    if not np.isnan(first_row['Real Estate']):
        re_row = first_row.copy()
        re_row['Asset_serial'] = 300
        re_row['Asset_Value'] = first_row['Real Estate']
        re_row['Asset_Name'] = 'Real Estate'
        rows.append(re_row)

    # 4. Gross wealth Row
    if not np.isnan(first_row['Gross wealth']):
        w_row = first_row.copy()
        w_row['Asset_serial'] = 1000
        w_row['Asset_Value'] = first_row['Gross wealth']
        w_row['Asset_Name'] = 'Gross wealth'
        rows.append(w_row)

    # 6. Total Financial Assets Row
    if not np.isnan(first_row['Total Financial Assets']):
        tfa_row = first_row.copy()
        tfa_row['Asset_serial'] = 1500
        tfa_row['Asset_Value'] = first_row['Total Financial Assets']
        tfa_row['Asset_Name'] = 'Total Financial Assets'
        rows.append(tfa_row)   

# Convert to DataFrame
asset_df = pd.DataFrame(rows)

# Optional: Arrange columns nicely
cols_order = group_cols + ['Asset_serial', 'Asset_Value'] + [col for col in df_grouped2.columns if col not in group_cols + ['Asset_serial', 'Asset_Value']]
asset_df = asset_df[cols_order]

# Check result
print(asset_df.head())



# Renaming Liabilities

asset_df = asset_df.rename(columns={
    'Credit_Agency': 'Liability_serial',
    'Liability': 'Liability_name'
})

asset_df.to_stata("final(28).dta", write_index=False)


        HHID  Credit_Agency  Fin_Asset_Serial  Share_Asset_serial  \
0  501171011           91.0                31                   5   
1  501171011           91.0                31                   5   
2  501171011           91.0                31                   5   
3  501171011           91.0                31                   5   
4  501171011           91.0                31                   5   

   Asset_serial  Asset_Value  State   MCE  Weight  Fin_Asset_Value  \
0            31       3000.0     24  3200  95.815           3000.0   
1           205        500.0     24  3200  95.815           3000.0   
2           300    1500000.0     24  3200  95.815           3000.0   
3          1000    1523500.0     24  3200  95.815           3000.0   
4          1500      23500.0     24  3200  95.815           3000.0   

   Share_Asset_Value  Liability_value  Real Estate Fin_Asset  \
0              500.0          10000.0    1500000.0  Currency   
1              500.0          10000.

C:\Users\siddu\AppData\Local\Temp\ipykernel_46848\347409114.py:69: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  asset_df.to_stata("final(28).dta", write_index=False)


In [401]:
group_cols = ['HHID', 'Liability_serial', 'Fin_Asset_Serial', 'Share_Asset_serial']
group_cols1 = ['HHID', 'Liability_serial', 'Fin_Asset_Serial', 'Share_Asset_serial', 'Asset_serial']

# Count unique combinations
num_subsets = asset_df[group_cols].drop_duplicates().shape[0]

print(f"Number of unique subsets: {num_subsets}")

# Check for duplicates
dup_check = asset_df.duplicated(subset=group_cols)
print(f"Number of duplicates: {dup_check.sum()}")

# Check for duplicates
dup_check = asset_df.duplicated(subset=group_cols1)
print(f"Number of duplicates: {dup_check.sum()}")

Number of unique subsets: 8062
Number of duplicates: 32248
Number of duplicates: 0


In [402]:
asset_df.dtypes

HHID                                object
Liability_serial                   float64
Fin_Asset_Serial                     int64
Share_Asset_serial                   int64
Asset_serial                         int64
Asset_Value                        float64
State                                int64
MCE                                  int64
Weight                             float64
Fin_Asset_Value                    float64
Share_Asset_Value                  float64
Liability_value                    float64
Real Estate                        float64
Fin_Asset                           object
Share_Asset                         object
Liability_name                      object
Gross wealth                       float64
Total Financial Assets             float64
Total Institutional liabilities    float64
Asset_Name                          object
dtype: object

In [403]:
# Check for missing values

# Check how many missing in each column
print(asset_df[['Asset_serial', 'Asset_Value', 'Asset_Name']].isna().sum())

# Optionally, view rows with missing in any of them
missing_rows = asset_df[
    asset_df[['Asset_serial', 'Asset_Value', 'Asset_Name']].isna().any(axis=1)
]

print(f"\nTotal rows with missing values: {len(missing_rows)}")
print(missing_rows.head())


Asset_serial    0
Asset_Value     0
Asset_Name      0
dtype: int64

Total rows with missing values: 0
Empty DataFrame
Columns: [HHID, Liability_serial, Fin_Asset_Serial, Share_Asset_serial, Asset_serial, Asset_Value, State, MCE, Weight, Fin_Asset_Value, Share_Asset_Value, Liability_value, Real Estate, Fin_Asset, Share_Asset, Liability_name, Gross wealth, Total Financial Assets, Total Institutional liabilities, Asset_Name]
Index: []


In [404]:
final_df = asset_df.copy()

In [405]:
final_df.columns

Index(['HHID', 'Liability_serial', 'Fin_Asset_Serial', 'Share_Asset_serial',
       'Asset_serial', 'Asset_Value', 'State', 'MCE', 'Weight',
       'Fin_Asset_Value', 'Share_Asset_Value', 'Liability_value',
       'Real Estate', 'Fin_Asset', 'Share_Asset', 'Liability_name',
       'Gross wealth', 'Total Financial Assets',
       'Total Institutional liabilities', 'Asset_Name'],
      dtype='object')

In [406]:
final_df.drop(['Fin_Asset_Serial', 
               'Share_Asset_serial', 
               'Fin_Asset_Value',
                'Share_Asset_Value', 
                'Fin_Asset',
                'Share_Asset',               
                ], axis=1, inplace=True)

final_df.to_stata("final(2019).dta", write_index=False)

C:\Users\siddu\AppData\Local\Temp\ipykernel_46848\486095085.py:9: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Real Estate   ->   Real_Estate
    Gross wealth   ->   Gross_wealth
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  final_df.to_stata("final(2019).dta", write_index=False)


In [407]:
final_df.columns

Index(['HHID', 'Liability_serial', 'Asset_serial', 'Asset_Value', 'State',
       'MCE', 'Weight', 'Liability_value', 'Real Estate', 'Liability_name',
       'Gross wealth', 'Total Financial Assets',
       'Total Institutional liabilities', 'Asset_Name'],
      dtype='object')

In [408]:
final_df = final_df[['HHID', 'State', 'MCE' , 'Weight','Gross wealth',  
                    'Liability_serial', 'Liability_name','Liability_value',
                    'Asset_serial', 'Asset_Name', 'Asset_Value',
                    'Real Estate', 'Total Financial Assets', 'Total Institutional liabilities', 
                    ]].copy()

final_df.to_stata("final(2019).dta", write_index=False)

final_df['HHID'].nunique()

C:\Users\siddu\AppData\Local\Temp\ipykernel_46848\340742307.py:7: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Gross wealth   ->   Gross_wealth
    Real Estate   ->   Real_Estate
    Total Financial Assets   ->   Total_Financial_Assets
    Total Institutional liabilities   ->   Total_Institutional_liabilities

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  final_df.to_stata("final(2019).dta", write_index=False)


1004

In [409]:
final_df1 = final_df[['HHID', 'State', 'MCE', 'Weight', 'Gross wealth',  
                    'Liability_serial', 'Liability_name','Liability_value',
                    'Asset_serial', 'Asset_Name', 'Asset_Value'
                    ]].copy()

final_df1.to_stata("final(2019)_1.dta", write_index=False)

final_df1['HHID'].nunique()

C:\Users\siddu\AppData\Local\Temp\ipykernel_46848\1597902558.py:6: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Gross wealth   ->   Gross_wealth

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  final_df1.to_stata("final(2019)_1.dta", write_index=False)


1004

In [410]:
final_df1.columns

Index(['HHID', 'State', 'MCE', 'Weight', 'Gross wealth', 'Liability_serial',
       'Liability_name', 'Liability_value', 'Asset_serial', 'Asset_Name',
       'Asset_Value'],
      dtype='object')

In [411]:
print(sorted(final_df1['Asset_serial'].unique()))

[31, 32, 34, 35, 36, 37, 205, 300, 1000, 1500]


In [412]:
group_cols1 = ['HHID', 'Liability_serial', 'Asset_serial']

In [413]:
final_df2 = final_df1.drop_duplicates(subset=group_cols1, keep='first').reset_index(drop=True)


In [414]:
final_df2.to_stata("final(2019)_2.dta", write_index=False)

C:\Users\siddu\AppData\Local\Temp\ipykernel_46848\2964308011.py:1: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Gross wealth   ->   Gross_wealth

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  final_df2.to_stata("final(2019)_2.dta", write_index=False)


In [415]:
# Check for repetition

# Check for uniqueness 

duplicates = final_df2.duplicated(subset=['Asset_serial', 'HHID', 'Liability_serial'], keep=False)

# Check if there are any duplicates
if duplicates.any():
    print("Some (Asset_serial, HHID) combinations are repeated:")
    print(final_df2.loc[duplicates, ['Asset_serial', 'HHID']].sort_values(['Asset_serial', 'HHID']))
else:
    print(" All HHIDs are unique within each Asset_serial category.")

# Check if there are any duplicates
if duplicates.any():
    print("Some (Liability_serial, HHID) combinations are repeated:")
    print(final_df2.loc[duplicates, ['Liability_serial', 'HHID']].sort_values(['Liability_serial', 'HHID']))
else:
    print(" All HHIDs are unique within each Liability_serial category.")


 All HHIDs are unique within each Asset_serial category.
 All HHIDs are unique within each Liability_serial category.


In [416]:
final_df2['HHID'].nunique()

1004

# Add State names corresponding to their codes

In [417]:
# final_df2 (Merge D&D, D&N)
final_df2.loc[final_df2['State'].isin([25,26]), 'State'] = 25
final_df2['State'].value_counts()

# State Map

State_map = {
    1.0: 'Jammu and Kashmir',
    2.0: 'Himachal Pradesh',
    3.0: 'Punjab',
    4.0: 'Chandigarh',
    5.0: 'Uttarakhand',
    6.0: 'Haryana',
    7.0: 'Delhi',
    8.0: 'Rajasthan',
    9.0: 'Uttar Pradesh',
    10.0: 'Bihar',
    11.0: 'Sikkim',
    12.0: 'Arunachal Pradesh',
    13.0: 'Nagaland',
    14.0: 'Manipur',
    15.0: 'Mizoram',
    16.0: 'Tripura',
    17.0: 'Meghalaya',
    18.0: 'Assam',
    19.0: 'West Bengal',
    20.0: 'Jharkhand',
    21.0: 'Odisha',
    22.0: 'Chhattishgarh',
    23.0: 'Madhya Pradesh',
    24.0: 'Gujarat',
    25.0: 'Daman and Diu and Dadra and Nagar Haveli',
    27.0: 'Maharashtra',
    28.0: 'Andhra Pradesh',
    29.0: 'Karnataka',
    30.0: 'Goa',
    31.0: 'Lakshadweep',
    32.0: 'Kerala',
    33.0: 'Tamilnadu',
    34.0: 'Puducherry',
    35.0: 'Andaman & Nicobar',
    36.0: 'Telengana'
}

final_df3 = final_df2.copy()

final_df3['State_name'] = final_df3['State'].map(State_map)

final_df3.to_stata('2019_final.dta', write_index=False)

C:\Users\siddu\AppData\Local\Temp\ipykernel_46848\2889506323.py:49: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    Gross wealth   ->   Gross_wealth

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  final_df3.to_stata('2019_final.dta', write_index=False)
